# Global DEB model parameters
## Parameters are estimated across all collected studies

### Initial e is a function of total soil organic carbon concentration

This analysis is performed in order to determine generalizability of DEB model.

### Importing libraries

In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution
from skopt import gp_minimize

### Importing modules
Modules calculate weighted residual sum of squares. One module is one study

In [2]:
from objectiveWP import objectiveWP
from objectiveM import objectiveM
from objectiveG import objectiveG
from objectiveB import objectiveB
#from objectiveZ import objectiveZ - do not estimate conversion to PLFA (only one study and two parameters) 
from objectiveBl import objectiveBl
from objectiveS import objectiveS
from objectiveJ import objectiveJ
from objectiveT import objectiveT
from objectiveN_A import objectiveN_A
from objectiveN_B import objectiveN_B
from objectiveN_C import objectiveN_C
from objectiveTS import objectiveTS

# List of parameters
0 - $y_{A}$  $~~~~~~~~~$   1 - $K_{M}$  $~~~~~~~~~$    2 - $v$   $~~~~~~~~~$   3 - $m$   $~~~~~~~~~$   4 - $g$ $~~~~~~~~~$ 5 - ce

6 - $n_{X_{1}}$ $~~~~~~~~~$ 7 - $i_{X_{1}}$ $~~~~~~~~~$ 8 - $t_{e}$ $~~~~~~~~~$ 9 - $t_{X_{1}}$ $~~~~~~~~~$ 10 - $MW_{X_{1}}$ $~~~~~~~~~$ 11 - $MW_{e}$

12 - $e_{Wardle}$ $~~~~~~~~~$ 13 - $DOC_{0}$ $~~~~~~~~~$ 14 - $DOCtoC_{TOT}$


## Objective function

It is simply the function calculating sum of weighted residual sum of squares returned from imported modules.

In [3]:
def objectiveGlobal (x):
    out = sum([objectiveWP(x), objectiveG(x), objectiveN_C(x), objectiveM(x), objectiveS(x), objectiveB(x), objectiveBl(x), 
     objectiveT(x), objectiveJ(x), objectiveN_A(x), objectiveN_B(x), objectiveTS(x)])
    return out

## Parameters estimation

Limits of model parameters are defined according to parameters estimated for individual studies.

In [4]:
#Individual parameters
yA = (0.05, 1)           
Km = (12, 4000)         
v = (0.03, 30)           
m = (1e-12, 0.1)           
g = (0.08, 5)          
ce = (0.3, 7)
nX1 = (0.1, 0.45)          
iX1 = (0.01, 0.2)           
te = (0.0001, 0.2)           
tX1 = (0.001, 0.05)           
MwX1 = (10, 500)          
Mwe = (10, 500)
eW = (0, 1)        
DOC0 = (0.001, 100)          
DOCtoCtot = (0.00001, 1)

#Parameter space
parSpace = [yA, Km, v, m, g, ce, nX1, iX1, te, tX1, MwX1, Mwe, 
            eW, DOC0, DOCtoCtot]

#### Using gaussian process

In [5]:
parGlobalGP = gp_minimize(objectiveGlobal, parSpace, n_calls = 10000)

/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/usr/local/lib/pytho

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
print(parGlobalGP.x)

In [ ]:
print(parGlobalGP)
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/GlobalSearch/parGlobalGPGeneralized.csv', parGlobalGP.x.reshape(1,15), delimiter=",")

#### Using dual annealing 

In [ ]:
parGlobalDA = dual_annealing(objectiveGlobal, parSpace, maxiter = 10000)

/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/GlobalSearchGeneralized/DEBmodelIso.py:17: RuntimeWarning: divide by zero encountered in double_scalars
  eatm = el/(el + eu)
/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/GlobalSearchGeneralized/DEBmodelIso.py:29: RuntimeWarning: invalid value encountered in double_scalars
  dCO2ldt = uptake*(1 - yA) + ce*((X1l + X1u)*(el + eu)*(v-growth))*eatm - max(0, (X1l + X1u)*growth*eatm)*MX1 - min(0, (X1l + X1u)*growth*X1atm)*MX1
/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/GlobalSearchGeneralized/DEBmodelIso.py:28: RuntimeWarning: invalid value encountered in double_scalars
  dX1ldt = max(0, (X1l + X1u)*growth

In [ ]:
print(parGlobalDA)
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/GlobalSearch/parGlobalDAGeneralized.csv', parGlobalDA.x.reshape(1,15), delimiter=",")

#### Differential evolution algorithm

In [ ]:
parGlobalDE = differential_evolution(objectiveGlobal, parSpace)

In [ ]:
print(parGlobalDE)
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/GlobalSearch/parGlobalDEGeneralized.csv', parGlobalDE.x.reshape(1,15), delimiter=",")

## Compare simulations with observations 

### Importing modules

In [ ]:
from FitWP import FitWP
from FitM import FitM
from FitG import FitG
from FitB import FitB
#from objectiveZ import objectiveZ - do not estimate conversion to PLFA (only one study and two parameters) 
from FitBl import FitBl
from FitS import FitS
from FitJ import FitJ
from FitT import FitT
from FitN_A import FitN_A
from FitN_B import FitN_B
from FitN_C import FitN_C
from FitTS import FitTS

#### Exporting data

In [ ]:
#DA
globalResultsDA = pd.concat([FitWP(parGlobalDA.x), FitM(parGlobalDA.x), FitG(parGlobalDA.x), FitB(parGlobalDA.x),
                            FitBl(parGlobalDA.x), FitS(parGlobalDA.x), FitJ(parGlobalDA.x), FitT(parGlobalDA.x),
                            FitN_A(parGlobalDA.x), FitN_B(parGlobalDA.x), FitN_C(parGlobalDA.x), FitTS(parGlobalDA.x)])
globalResultsDA.to_csv('globalResultsDAGeneralized.csv')
#DE
globalResultsDE = pd.concat([FitWP(parGlobalDE.x), FitM(parGlobalDE.x), FitG(parGlobalDE.x), FitB(parGlobalDE.x),
                            FitBl(parGlobalDE.x), FitS(parGlobalDE.x), FitJ(parGlobalDE.x), FitT(parGlobalDE.x),
                            FitN_A(parGlobalDE.x), FitN_B(parGlobalDE.x), FitN_C(parGlobalDE.x), FitTS(parGlobalDE.x)])
globalResultsDE.to_csv('globalResultsDEGeneralized.csv')
#GP
globalResultsGP = pd.concat([FitWP(parGlobalGP.x), FitM(parGlobalGP.x), FitG(parGlobalGP.x), FitB(parGlobalGP.x),
                            FitBl(parGlobalGP.x), FitS(parGlobalGP.x), FitJ(parGlobalGP.x), FitT(parGlobalGP.x),
                            FitN_A(parGlobalGP.x), FitN_B(parGlobalGP.x), FitN_C(parGlobalGP.x), FitTS(parGlobalGP.x)])
globalResultsGP.to_csv('globalResultsGPGeneralized.csv')